In [16]:
using SparseArrays

In [69]:
include("src/core_parser.jl")

parsemps_bounds! (generic function with 1 method)

In [2]:
include("src/time_parser.jl")

parse_time_file (generic function with 1 method)

# AIRL tests

## Parsing `.cor` file

In [74]:
cor_files = [f for f in readdir("tssp/") if f[end-3:end] == ".cor"]

12-element Array{String,1}:
 "4node.cor"   
 "AIRL.cor"    
 "LandS.cor"   
 "assets.cor"  
 "chem.cor"    
 "env.cor"     
 "fxm.cor"     
 "phone.cor"   
 "pltexpA2.cor"
 "stocfor1.cor"
 "stocfor2.cor"
 "stormG2.cor" 

In [106]:
d, row2idx, col2idx, I, J, V, cI, cV, bI, bV = parse_cor_file("AIRL.cor")

(Dict{String,Any}("name"=>"AIRL","nrows"=>8,"range"=>"","rhs"=>"RIGHT","ncols"=>12,"bound"=>""), Dict{Any,Any}("HOURS2"=>2,"OBJ"=>0,"AVAIL12"=>4,"HOURS1"=>1,"AVAIL21"=>5,"DEMAND1"=>7,"AVAIL11"=>3,"DEMAND2"=>8,"AVAIL22"=>6), Dict{Any,Any}("YMINUS2"=>12,"YPLUS2"=>10,"X121"=>6,"X21"=>3,"X212"=>7,"X11"=>1,"X22"=>4,"RIGHT"=>0,"YMINUS1"=>11,"YPLUS1"=>9…), [1, 3, 7, 1, 4, 8, 2, 5, 7, 2  …  5, 7, 8, 6, 8, 7, 7, 8, 7, 8], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  7, 7, 7, 8, 8, 8, 9, 10, 11, 12], [24.0, -24.0, 50.0, 14.0, -14.0, 75.0, 49.0, -49.0, 20.0, 29.0  …  36.0, -14.6939, 20.0, 56.0, -38.621, 20.0, 1.0, 1.0, -1.0, -1.0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [7200.0, 6000.0, 7200.0, 4000.0, 1300.0, -4228.57, 210.204, 975.862, 500.0, 250.0], [1, 2, 7, 8], [7200.0, 7200.0, 1.0, 1.0])

## Parsing `.tim` file

In [24]:
name, is_lp, col_periods, row_periods = parse_time_file("AIRL.tim")

("AIRL", false, ["X11", "X112"], ["HOURS1", "AVAIL11"])

In [29]:
col2idx[col_periods[2]]-1

4

In [30]:
row2idx[row_periods[2]]-1

2

In [40]:
first_period_vars = 1:(col2idx[col_periods[2]]-1);
second_period_vars = col2idx[col_periods[2]]:length(col2idx);

first_period_rows = 1:(row2idx[row_periods[2]]-1);
second_period_rows = row2idx[row_periods[2]]:(length(row2idx)-1);

In [41]:
first_period_vars, second_period_vars

(1:4, 5:12)

In [42]:
first_period_rows, second_period_rows

(1:2, 3:8)

In [45]:
Matrix(A_)

8×12 Array{Float64,2}:
  24.0   14.0    0.0    0.0    0.0    …    0.0    0.0  0.0   0.0   0.0
   0.0    0.0   49.0   29.0    0.0         0.0    0.0  0.0   0.0   0.0
 -24.0    0.0    0.0    0.0   19.0         0.0    0.0  0.0   0.0   0.0
   0.0  -14.0    0.0    0.0    0.0         0.0    0.0  0.0   0.0   0.0
   0.0    0.0  -49.0    0.0    0.0         0.0    0.0  0.0   0.0   0.0
   0.0    0.0    0.0  -29.0    0.0    …   56.0    0.0  0.0   0.0   0.0
  50.0    0.0   20.0    0.0  -39.583      20.0    1.0  0.0  -1.0   0.0
   0.0   75.0    0.0   20.0   75.0       -38.621  0.0  1.0   0.0  -1.0

In [46]:
A_ = sparse(I, J, V);
A = Matrix(A_[first_period_rows, first_period_vars])
T = Matrix(A_[second_period_rows, first_period_vars])
W = Matrix(A_[second_period_rows, second_period_vars])

6×8 Array{Float64,2}:
  19.0       0.0      0.0       0.0    0.0  0.0   0.0   0.0
   0.0      29.0      0.0       0.0    0.0  0.0   0.0   0.0
   0.0       0.0     36.0       0.0    0.0  0.0   0.0   0.0
   0.0       0.0      0.0      56.0    0.0  0.0   0.0   0.0
 -39.583    50.0    -14.6939   20.0    1.0  0.0  -1.0   0.0
  75.0    -155.357   20.0     -38.621  0.0  1.0   0.0  -1.0

In [47]:
A

2×4 Array{Float64,2}:
 24.0  14.0   0.0   0.0
  0.0   0.0  49.0  29.0

In [48]:
T

6×4 Array{Float64,2}:
 -24.0    0.0    0.0    0.0
   0.0  -14.0    0.0    0.0
   0.0    0.0  -49.0    0.0
   0.0    0.0    0.0  -29.0
  50.0    0.0   20.0    0.0
   0.0   75.0    0.0   20.0

In [49]:
W

6×8 Array{Float64,2}:
  19.0       0.0      0.0       0.0    0.0  0.0   0.0   0.0
   0.0      29.0      0.0       0.0    0.0  0.0   0.0   0.0
   0.0       0.0     36.0       0.0    0.0  0.0   0.0   0.0
   0.0       0.0      0.0      56.0    0.0  0.0   0.0   0.0
 -39.583    50.0    -14.6939   20.0    1.0  0.0  -1.0   0.0
  75.0    -155.357   20.0     -38.621  0.0  1.0   0.0  -1.0

## Parsing `.sto` file

In [71]:
include("src/stoch_parser.jl")

parse_block_line! (generic function with 1 method)

In [72]:
sto_files = [f for f in readdir("tssp/") if occursin(".sto", f)];

In [75]:
length(sto_files)

49

In [79]:
n_large = 0

for f in sto_files
    
    try
        name, indeps, blocks = parse_sto_file("tssp/"*f)
        

        # Check if some blocms must be compared to ref block
        for br in blocks
            realizations = br[2]

            ref = realizations[1].values

            for r_ in realizations[2:end]
                # check whether some keys are in ref but not in current block
                s_ = setdiff(keys(ref), keys(r_.values))
                length(s_) > 0 && println("\t", f, "\n", br[1])
            end
        end
        
        # Compute all scenarios
        S_ = Base.Iterators.product(values(indeps)..., values(blocks)...)
        S = [prod(collect(s)) for s in S_]
        length(S) >= 1000 && println(name, "\n\t|S| = $(length(S))")
        if length(S) >= 1000
            n_large += 1
        end
        
    catch err
        println(f, "\tError: $err")
    end
    
end

4NODECAR
	|S| = 1024
4NODECAR
	|S| = 16384
4NODECAR
	|S| = 2048
4NODECAR
	|S| = 32768
4NODECAR
	|S| = 4096
4NODECAR
	|S| = 8192
ASS2BY5
	|S| = 37500
ENV
	|S| = 1200
ENV
	|S| = 1875
ENV
	|S| = 32928
ENV
	|S| = 3780
ENV
	|S| = 5292
ENV
	|S| = 8232
fxmev.sto	Error: MethodError(Base.reduce_empty, (Base.mul_prod, Union{}), 0x000000000000620c)
PHONE
	|S| = 32768
stocfor1.sto	Error: MethodError(Base.reduce_empty, (Base.mul_prod, Union{}), 0x000000000000620c)
storm
	|S| = 1000


In [15]:
name, indeps, blocks = parse_sto_file("tssp/env.sto.15");

In [48]:
for br in blocks
    realizations = br[2]
    
    ref = realizations[1].values
    
    for r_ in realizations[2:end]
        # check whether some keys are in ref but not in current block
        s_ = setdiff(keys(ref), keys(r_.values))
        println(length(s_))
    end
end

0
0


In [20]:
S_ = Base.Iterators.product(values(indeps)..., values(blocks)...)
S = [prod(collect(s)) for s in S_]
length(S)

15

In [25]:
for s in S
    display(s.values)
    println()
end

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 550.0
  ("CO2LIM2", "IMELC2WN") => 1.2
  ("CO2LIM2", "IMELC2SD") => 1.2
  ("CO2LIM2", "IMELC2SN") => 1.2
  ("CO2LIM2", "IMELC2WD") => 1.2

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 470.0
  ("CO2LIM2", "IMELC2WN") => 1.2
  ("CO2LIM2", "IMELC2SD") => 1.2
  ("CO2LIM2", "IMELC2SN") => 1.2
  ("CO2LIM2", "IMELC2WD") => 1.2

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 460.0
  ("CO2LIM2", "IMELC2WN") => 1.2
  ("CO2LIM2", "IMELC2SD") => 1.2
  ("CO2LIM2", "IMELC2SN") => 1.2
  ("CO2LIM2", "IMELC2WD") => 1.2

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 420.0
  ("CO2LIM2", "IMELC2WN") => 1.2
  ("CO2LIM2", "IMELC2SD") => 1.2
  ("CO2LIM2", "IMELC2SN") => 1.2
  ("CO2LIM2", "IMELC2WD") => 1.2

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 410.0
  ("CO2LIM2", "IMELC2WN") => 1.2
  ("CO2LIM2", "IMELC2SD") => 1.2
  ("CO2LIM2", "IMELC2SN") => 1.2
  ("CO2LIM2", "IMELC2WD") => 1.2

...

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 550.0
  ("CO2LIM2", "IMELC2WN") => 0.0
  ("CO2LIM2", "IMELC2SD") => 0.0
  ("CO2LIM2", "IMELC2SN") => 0.0
  ("CO2LIM2", "IMELC2WD") => 0.0

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 470.0
  ("CO2LIM2", "IMELC2WN") => 0.0
  ("CO2LIM2", "IMELC2SD") => 0.0
  ("CO2LIM2", "IMELC2SN") => 0.0
  ("CO2LIM2", "IMELC2WD") => 0.0

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 460.0
  ("CO2LIM2", "IMELC2WN") => 0.0
  ("CO2LIM2", "IMELC2SD") => 0.0
  ("CO2LIM2", "IMELC2SN") => 0.0
  ("CO2LIM2", "IMELC2WD") => 0.0

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 420.0
  ("CO2LIM2", "IMELC2WN") => 0.0
  ("CO2LIM2", "IMELC2SD") => 0.0
  ("CO2LIM2", "IMELC2SN") => 0.0
  ("CO2LIM2", "IMELC2WD") => 0.0

Dict{Tuple{String,String},Float64} with 5 entries:
  ("CO2LIM2", "RIGHT")    => 410.0
  ("CO2LIM2", "IMELC2WN") => 0.0
  ("CO2LIM2", "IMELC2SD") => 0.0
  ("CO2LIM2", "IMELC2SN") => 0.0
  ("CO2LIM2", "IMELC2WD") => 0.0